# Install dependencies

In [ ]:
# !pip install Keras==2.1.6 pydot tensorflow-gpu==1.8.0 
# !sudo apt-get install graphviz -y

# Imports

In [5]:
# from keras.utils.vis_utils import model_to_dot
import pydot
from IPython.display import SVG
import json
import numpy as np
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.utils import multi_gpu_model
from keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
# import pandas as pd
# from sklearn.metrics import confusion_matrix
import itertools
import shutil
import os
# import matplotlib.pyplot as plt

# %matplotlib inline

# Setup

In [6]:
!ls data/images

apple_pie	    eggs_benedict	     onion_rings
baby_back_ribs	    escargots		     oysters
baklava		    falafel		     pad_thai
beef_carpaccio	    filet_mignon	     paella
beef_tartare	    fish_and_chips	     pancakes
beet_salad	    foie_gras		     panna_cotta
beignets	    french_fries	     peking_duck
bibimbap	    french_onion_soup	     pho
bread_pudding	    french_toast	     pizza
breakfast_burrito   fried_calamari	     pork_chop
bruschetta	    fried_rice		     poutine
caesar_salad	    frozen_yogurt	     prime_rib
cannoli		    garlic_bread	     pulled_pork_sandwich
caprese_salad	    gnocchi		     ramen
carrot_cake	    greek_salad		     ravioli
ceviche		    grilled_cheese_sandwich  red_velvet_cake
cheesecake	    grilled_salmon	     risotto
cheese_plate	    guacamole		     samosa
chicken_curry	    gyoza		     sashimi
chicken_quesadilla  hamburger		     scallops
chicken_wings	    hot_and_sour_soup	     seaweed_salad
chocolate_cake	    hot_dog		     shrimp_and_grits
chocolate_mousse    huevo

In [7]:
NUM_GPUS = 1
train_folder = "data/train"
valid_folder = "data/valid"

In [8]:
!ls data/

images	train  valid


In [9]:
IMAGE_SIZE=224 
BATCH_SIZE=64 # number of images seen by model at once

In [10]:
!ls data/train/ | head

apple_pie
baby_back_ribs
baklava
beef_carpaccio
beef_tartare
beet_salad
beignets
bibimbap
bread_pudding
breakfast_burrito


In [11]:
train_generator = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input
                                     ,rotation_range=45
                                     ,width_shift_range=0.2
                                     ,height_shift_range=0.2
                                     ,shear_range=0.2
                                     ,zoom_range=0.25
                                     ,horizontal_flip=True
                                     ,fill_mode='nearest'
                                  )

train_batches = train_generator.flow_from_directory(
    train_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE,shuffle=True, seed=13,class_mode='categorical')



valid_generator =  ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)


valid_batches = valid_generator.flow_from_directory(
    valid_folder, target_size=(IMAGE_SIZE,IMAGE_SIZE), batch_size=BATCH_SIZE)

Found 80800 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [12]:
NUM_CLASSES = train_batches.num_classes

In [13]:
base_model = keras.applications.mobilenet.MobileNet()

In [14]:
?keras.applications.mobilenet.MobileNet()

In [15]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [ ]:
SVG(model_to_dot(base_model,show_shapes=True, show_layer_names=True).create(prog='dot', format='svg'))

In [ ]:
base_model.layers[-6].output

In [16]:
x = base_model.layers[-6].output
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [17]:
model = Model(inputs=base_model.input, outputs=predictions)


In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [19]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [20]:
optimizer = RMSprop(lr=0.001, rho=0.9)

#optimizer = Adam(lr=0.001)

# model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

#model.compile(Adam(lr=.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
?multi_gpu_model

In [21]:
try:
    model = multi_gpu_model(model, gpus=NUM_GPUS)
except:
    pass
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
steps_per_epoch = train_batches.n//BATCH_SIZE
validation_steps = valid_batches.n // BATCH_SIZE

# steps_per_epoch = 10
# validation_steps = 3

In [23]:
!rm -rf logs

In [24]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1)

tb = keras.callbacks.TensorBoard(log_dir='./logs', write_graph=True, write_images=True)


In [26]:
model.fit_generator(train_batches, steps_per_epoch=steps_per_epoch 
                    , validation_data=valid_batches
                    ,  validation_steps=validation_steps, verbose=2,workers=4 
                    , use_multiprocessing=True
                    , epochs=3
                   # , callbacks = [tb]
                    , callbacks = [reduce_lr, tb]
                   )

Epoch 1/3


Process PoolWorker-19:
Process PoolWorker-18:
Process PoolWorker-21:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Process PoolWorker-23:
Process PoolWorker-17:
Process PoolWorker-20:
Process PoolWorker-22:
Process PoolWorker-24:
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
    self.run()
Traceback (most recent call last):
    self.run()
    self.run()
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/multiproces

Epoch 1/3


  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/site-packages/keras_preprocessing/image.py", line 1296, in __getitem__
KeyboardInterrupt
    return self._get_batches_of_transformed_samples(index_array)
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/site-packages/keras_preprocessing/image.py", line 1772, in _get_batches_of_transformed_samples
    interpolation=self.interpolation)
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/site-packages/keras_preprocessing/image.py", line 495, in load_img
    img = pil_image.open(path)
  File "/home/alexandre.saez/anaconda3/envs/tf2/lib/python2.7/site-packages/PIL/Image.py", line 1956, in open
    prefix = fp.read(16)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
train_batches.class_indices.keys()

In [ ]:
train_batches.class_indices

In [ ]:
MODEL_SAVE_DIR = f'model/{DATASET_NAME}'

In [ ]:
!mkdir -p {MODEL_SAVE_DIR}

In [ ]:
shutil.rmtree(MODEL_SAVE_DIR)
os.makedirs(MODEL_SAVE_DIR)

In [ ]:
model.save(f'{MODEL_SAVE_DIR}/model.h5')

In [ ]:
with open(f'{MODEL_SAVE_DIR}/labels.txt', 'w') as file_handler:
    for item in train_batches.class_indices.keys():
        file_handler.write("{}\n".format(item))

In [ ]:
with open(f'{MODEL_SAVE_DIR}/labels.json', 'w') as outfile:
    json.dump(train_batches.class_indices, outfile)